In [0]:
spark

In [0]:
lookup_rules ={
    "rule1": "show_id is NOT NULL"
}

In [0]:
%MD
DLT TABLE GOLD LAYER

In [0]:
#define a streaming table to ingest data from a volume
@dlt.table(
    name= "gold_netflixdirectors"

)
#we can use this as well
@dlt.expect_or_drop("rule1", "show_id is NOT NULL")
def myfunc():
    df = spark.readStream.format("delta").load("abfss://silver@datalakenetflixsuresh.dfs.core.windows.net/netflix_directors")
    return df

In [0]:
@dlt.table(
    name = "gold_netflixcast"
)
@dlt.expect_or_drop(lookup_rules)
def myfunc():
    df = spark.readStream.format("delta").load("abfss://silver@datalakenetflixsuresh.dfs.core.windows.net/netflix_cast")
    return df

In [0]:
@dlt.table(
    name = "gold_netflixcountries"
)
@dlt.expect_or_drop(lookup_rules)
def myfunc():
    df = spark.readStream.format("delta").load("abfss://silver@datalakenetflixsuresh.dfs.core.windows.net/netflix_countries")
    return df

In [0]:
@dlt.table(
    name = "gold_netflixcategories"
)
@dlt.expect_or_drop(lookup_rules)
def myfunc():
    df = spark.readStream.format("delta").load("abfss://silver@datalakenetflixsuresh.dfs.core.windows.net/netflix_categories")
    return df

In [0]:
@dlt.table

def gold_stg_netflixfiles():
    df = spark.readStream.format("delta").load("abfss://silver@datalakenetflixsuresh.dfs.core.windows.net/netflix_files")
    return df




In [0]:
@dlt.view

def gold_transform_netflixfiles():
    df = spark.readStream.table("LIVE.gold_stg_netflixfiles")
    df = df.withColumn("newflag",lit(1))
    return df


In [0]:
masterdata_rules ={
    "rule1": "newflag is NOT NULL",
    "rule2": "show_id id NOT NULL"
}

In [0]:
@dlt.table
@dlt.expect_all_or_drop(masterdata_rules)
def gold_netflixfiles():
    df = spark.readStream.table("LIVE.gold_transform_netflixfiles")
    return df